In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [24]:
bnf_1min = pd.read_csv("../data/bn_30m.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [25]:
bnf_1min.tail()

,time,open,high,low,close,MA,MA.1,EMA,datetime
22664,2025-01-24T09:15:00+05:30,48582.3000,48720.2500,48468.8000,48518.7000,48601.9542,48737.5750,48656.8164,2025-01-24 09:15:00+05:30
22665,2025-01-24T09:45:00+05:30,48523.9000,48546.0000,48349.0500,48451.1500,48566.0875,48708.6833,48644.3518,2025-01-24 09:45:00+05:30
22666,2025-01-24T10:15:00+05:30,48441.1000,48621.6000,48347.8500,48587.4000,48542.5333,48690.1208,48640.9002,2025-01-24 10:15:00+05:30
22667,2025-01-24T10:45:00+05:30,48585.0500,48816.7000,48536.0500,48796.3000,48537.3333,48695.6958,48650.3183,2025-01-24 10:45:00+05:30
22668,2025-01-24T11:15:00+05:30,48790.3500,48813.2500,48692.6500,48737.8000,48548.4208,48708.8083,48655.6203,2025-01-24 11:15:00+05:30


In [26]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [27]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2019, 8, 8),
 datetime.date(2024, 10, 30),
 datetime.date(2019, 5, 9),
 datetime.date(2022, 4, 27),
 datetime.date(2024, 2, 23),
 datetime.date(2023, 6, 13),
 datetime.date(2023, 6, 19),
 datetime.date(2019, 3, 1),
 datetime.date(2022, 3, 4),
 datetime.date(2023, 2, 9),
 datetime.date(2021, 2, 22),
 datetime.date(2020, 1, 7),
 datetime.date(2019, 8, 27),
 datetime.date(2023, 11, 20),
 datetime.date(2022, 7, 13),
 datetime.date(2022, 8, 1),
 datetime.date(2024, 11, 4),
 datetime.date(2023, 1, 6),
 datetime.date(2020, 11, 18),
 datetime.date(2023, 3, 6),
 datetime.date(2018, 1, 16),
 datetime.date(2019, 5, 13),
 datetime.date(2020, 12, 15),
 datetime.date(2021, 1, 20),
 datetime.date(2021, 7, 15),
 datetime.date(2024, 4, 12),
 datetime.date(2018, 3, 16),
 datetime.date(2020, 8, 13),
 datetime.date(2020, 10, 27),
 datetime.date(2020, 3, 23),
 datetime.date(2018, 6, 26),
 datetime.date(2020, 11, 26),
 datetime.date(2023, 11, 9),
 datetime.date(2023, 4, 21),
 datetime.date(20

In [28]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [29]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0001
LEVERAGE_ = 5

In [30]:
def calculate_signals(df, ema_length, x_days, pct):
    # Calculate X-day High/Low and 20 EMA
    df['X_High'] = df['high'].rolling(x_days).mean()
    df['X_Low'] = df['low'].rolling(x_days).mean()
    df['EMA_20'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Trend conditions
    df['EMA_Trend'] = df['EMA_20'] < df['EMA_20'].shift(1)
    
    # Signal conditions
    df['Sell_Signal'] = (
        (df['close'] < df['X_Low'].shift(1) * (1 - (pct / 100))) & 
        (df['EMA_Trend']) & 
        (df['close'] < df['EMA_20'])
    )
    
    return df

In [31]:
def backtest3(df):
    # print(df.to_string())
    # return
    df.reset_index(drop=True, inplace=True)
    # Variables for short trades
    short_position = 0  # 0 = no position, 1 = short
    short_entry_price = 0
    short_entry_date = None
    short_trades = []
    tradebook = pd.DataFrame()
    tradebook_short = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    short_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_short = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for short trades
        short_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Sell_Signal'] and short_position == 0:
            short_position = 1
            short_entry_price = df.loc[i, 'close']
            short_entry_date = df.loc[i, 'datetime']
            short_trailing_stop = df.loc[i, 'EMA_20']
            short_initial_sl = df.loc[i, 'X_High']

            tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'short',
                'Entry_Price': df.loc[i, 'close'],
                'Previous MA Value': df.loc[i-1, 'X_Low'],
                'Initial SL': short_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = short_initial_sl
                lock_initial_sl = True

            can_pyramid = True
            pyramid_number = 0
            pyramid_high = 0
            continue
                
        # Exit or add to short position
        if short_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'high'] >= short_initial_sl:
                if df.loc[i, 'open'] >= short_initial_sl:
                    tradebook_short.loc[
                        (tradebook_short['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , df.loc[i, 'open'], 'Gap']
                    short_position = 0
                    can_add_short = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
                    # if df.loc[i, 'high'] >= long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , long_initial_sl, 'Initial SL Hit']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                    # elif df.loc[i, 'high'] < long_initial_sl:
                    #     tradebook_long.loc[
                    #         (tradebook_long['Trade No.'] == trade_number),
                    #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    #     ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'], 'Gap Open-Close Below ISL']
                    #     long_position = 0
                    #     can_add_long = False
                    #     trade_number += 1
                    #     lock_initial_sl = False
                    #     first_sl = 0
                    #     can_pyramid = False
                    #     pyramid_number = 0
                    #     pyramid_high = 0
                else:
                    tradebook_short.loc[
                        (tradebook_short['Trade No.'] == trade_number),
                        ['Exit_Date', 'Exit_Price', 'Exit Remark']
                    ] = [df.loc[i, 'datetime'] , short_initial_sl, 'Initial SL Hit']
                    short_position = 0
                    can_add_short = False
                    trade_number += 1
                    lock_initial_sl = False
                    first_sl = 0
                    can_pyramid = False
                    pyramid_number = 0
                    pyramid_high = 0
            
            elif df.loc[i, 'close'] >= short_trailing_stop:
                tradebook_short.loc[
                    (tradebook_short['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                short_position = 0
                can_add_short = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            # elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
            #     # Previous Y Candle Low Breached, confirmation found. Add position on high break
            #     can_add_long = True

            # if can_add_long and long_position and can_pyramid:
            #     if df.loc[i, 'high'] >= y_days_high and pyramid_number < pyr_num:
            #         tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
            #             'Trade No.': trade_number,
            #             'Entry_Date': df.loc[i, 'datetime'],
            #             'Exit_Date': None,
            #             'Trade_Type': 'Add_Long',
            #             'Entry_Price': y_days_high,
            #             'Initial SL': first_sl,
            #             'Exit_Price': None,
            #             # 'Profit/Loss': None  # Filled when exited
            #         }])], ignore_index=True)
            #         can_add_long = False
            #         pyramid_number += 1
            #         can_pyramid = True
    
    tradebook = pd.concat([tradebook_short, tradebook], ignore_index=True)
    return tradebook


In [32]:
ema_length = 20
x_days = 26
# y_days = 4
pct = 0.1
RPT = 2.5
df = bnf_1min
df = calculate_signals(df, ema_length,x_days, pct)
# print(df.tail().to_string())
# df1 = df[df['Buy_Signal']]
# print(df1.to_string())
tb = backtest3(df)
tb = tb.sort_values(by='Entry_Date')
# variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_59288/532239134.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


In [33]:
import numpy as np
portfolio = 1000000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
# tb['Qty'] = portfolio * 5 / tb['Entry_Price']
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
tb['Trade Year'] = tb['Entry_Date'].dt.year

In [34]:
tb = tb.reset_index()

In [35]:
tb

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Previous MA Value,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
0,0,0,2018-01-10 09:15:00+05:30,2018-01-11 11:45:00+05:30,short,25647.8000,25674.9385,25718.8000,25628.2000,Trailing SL Hit,19.6000,5.1276,14.4724,352.1127,5095.9155,0.5096,2018
1,1,1,2018-01-12 12:15:00+05:30,2018-01-12 13:15:00+05:30,short,25565.0000,25621.9327,25668.2077,25668.2077,Initial SL Hit,-103.2077,5.1233,-108.3310,242.2300,-26241.0220,-2.6241,2018
2,2,2,2018-01-25 12:45:00+05:30,2018-01-25 13:45:00+05:30,short,27266.9000,27305.4923,27389.1769,27389.1769,Initial SL Hit,-122.2769,5.4656,-127.7425,204.4540,-26117.4651,-2.6117,2018
3,3,3,2018-01-30 09:15:00+05:30,2018-01-31 14:15:00+05:30,short,27328.5500,27406.5038,27504.4077,27398.5000,Trailing SL Hit,-69.9500,5.4727,-75.4227,142.1604,-10722.1220,-1.0722,2018
4,4,4,2018-02-01 12:15:00+05:30,2018-02-01 12:45:00+05:30,short,27173.6000,27272.5038,27364.5865,27364.5865,Initial SL Hit,-190.9865,5.4538,-196.4404,130.8993,-25713.9009,-2.5714,2018
5,5,5,2018-02-01 14:45:00+05:30,2018-02-06 14:15:00+05:30,short,27200.2000,27264.5981,27396.4212,25916.5500,Trailing SL Hit,1283.6500,5.3117,1278.3383,127.4073,162869.5862,16.2870,2018
6,6,6,2018-02-09 09:15:00+05:30,2018-02-12 10:15:00+05:30,short,25517.2500,25799.7827,25899.1942,25595.0500,Trailing SL Hit,-77.8000,5.1112,-82.9112,65.4546,-5426.9199,-0.5427,2018
7,7,7,2018-02-14 11:15:00+05:30,2018-02-15 09:45:00+05:30,short,25497.7000,25524.2192,25619.5538,25525.7500,Trailing SL Hit,-28.0500,5.1023,-33.1523,205.1638,-6801.6616,-0.6802,2018
8,8,8,2018-02-15 10:45:00+05:30,2018-02-15 12:15:00+05:30,short,25484.0000,25518.0731,25602.6865,25566.8500,Trailing SL Hit,-82.8500,5.1051,-87.9551,210.6389,-18526.7609,-1.8527,2018
9,9,9,2018-02-15 13:15:00+05:30,2018-02-16 09:15:00+05:30,short,25410.3000,25495.4288,25585.0846,25585.0846,Initial SL Hit,-174.7846,5.0995,-179.8842,143.0332,-25729.4032,-2.5729,2018


In [36]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [37]:
tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
tb['Trade Year'] = tb['Entry_Date'].dt.year
# tb['ROI% w cs'] = tb['ROI%']

In [38]:
stats = generate_stats(tb, 'Short')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,0,0,NaN,NaN,NaN,NaN,NaN,Short
2018,59.0480,87,33.3333,5.3600,-1.6620,-15.6659,3.7692,Short
2019,46.7169,97,37.1134,4.3054,-1.7750,-16.4015,2.8483,Short
2020,67.4973,107,34.5794,5.0128,-1.6854,-28.2625,2.3882,Short
2021,43.0948,101,30.6931,5.2696,-1.7180,-21.1597,2.0366,Short
2022,9.0805,101,21.7822,6.7395,-1.7619,-37.3206,0.2433,Short
2023,33.4430,86,25.5814,6.3018,-1.6437,-46.7303,0.7157,Short
2024,36.1814,85,38.8235,3.9932,-1.8384,-16.4566,2.1986,Short
Overall,295.0618,664,31.8386,5.1052,-1.7256,-46.7303,6.3141,Short


In [36]:
# tb.to_csv('Turtle-Shorting.csv', index=False)

In [22]:
# ema_length = 55
# x_days = 28
# y_days = 7
# pct = 1.05
# RPT = 5

pct_range = [0.1, 0.25, 0.33, 0.5]
RPT_ = 3
stats_dictionary = {}

for ema_length in range(8, 81, 2):
    for x_days in range(4, 37, 2):
            for pct in pct_range:
                # RPT = 5
                variation = f'EMA:{ema_length}, X:{x_days}, PCT:{pct}'
                print(variation)
                df = bnf_1min
                df = calculate_signals(df, ema_length=ema_length, x_days=x_days, pct=pct)
                tb = backtest3(df)
                if len(tb) > 0:
                    tb = tb.sort_values(by='Entry_Date')
                    portfolio = 10000000
                    tb['PnL'] = np.where(
                        tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                        tb['Exit_Price'] - tb['Entry_Price'],
                        np.where(
                            tb['Trade_Type'].isin(['short', 'Add_Short']),  # For Short and Add_Short
                            tb['Entry_Price'] - tb['Exit_Price'],
                            0  # Default case if Trade_Type is something unexpected
                        )
                    )
                    
                    tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                    tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                    tb['Qty'] = abs(RPT_ / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                    tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                    tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                    tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                    tb['Trade Year'] = tb['Entry_Date'].dt.year
                    # tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                    # tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                    # tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                    # tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()
                    
                    stats1 = generate_stats(tb, variation)
                    for x, y in stats1.items():
                        if x > 1:
                            # print('Long Only')
                            print(pd.DataFrame(y).to_string())
                            stats_dictionary[x] = y
                        break

EMA:8, X:4, PCT:0.1


/tmp/ipykernel_59288/532239134.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


EMA:8, X:4, PCT:0.25


/tmp/ipykernel_59288/532239134.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio             Variation
2017            0            0      NaN                   NaN                 NaN          NaN          NaN  EMA:8, X:4, PCT:0.25
2018      42.1477           79  34.1772                5.0091             -1.7904     -14.5837       2.8900  EMA:8, X:4, PCT:0.25
2019      16.8013           93  39.7849                3.1303             -1.7682     -24.7836       0.6779  EMA:8, X:4, PCT:0.25
2020      54.4498          117  37.6068                4.3247             -1.8608     -35.7385       1.5236  EMA:8, X:4, PCT:0.25
2021      50.3127           94  42.5532                4.0653             -2.0796     -21.5078       2.3393  EMA:8, X:4, PCT:0.25
2022     -21.4596          124  33.0645                3.2429             -1.8605     -61.0597      -0.3515  EMA:8, X:4, PCT:0.25
2023     -13.8768           74  31.0811                3.4455             -1.8260     -32.

...

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio               Variation
2017            0            0      NaN                   NaN                 NaN          NaN          NaN  EMA:80, X:36, PCT:0.25
2018      49.2405           45  31.1111                6.6459             -1.4130     -21.0837       2.3355  EMA:80, X:36, PCT:0.25
2019     -10.2893           56  19.6429                5.3981             -1.5482     -29.6025      -0.3476  EMA:80, X:36, PCT:0.25
2020      63.7401           45  22.2222               12.1401             -1.6474     -34.4132       1.8522  EMA:80, X:36, PCT:0.25
2021     -26.1335           54  18.5185                4.9696             -1.7234     -61.0163      -0.4283  EMA:80, X:36, PCT:0.25
2022      29.5564           55  21.8182                8.8484             -1.7820     -33.9295       0.8711  EMA:80, X:36, PCT:0.25
2023       0.8718           42  21.4286                6.2484             -1

/tmp/ipykernel_59288/532239134.py:118: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Initial SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio               Variation
2017            0            0      NaN                   NaN                 NaN          NaN          NaN  EMA:80, X:36, PCT:0.33
2018      49.6458           40  30.0000                7.3560             -1.3795     -17.0015       2.9201  EMA:80, X:36, PCT:0.33
2019      -7.9324           50  20.0000                5.6675             -1.6152     -26.1747      -0.3031  EMA:80, X:36, PCT:0.33
2020      68.9744           43  23.2558               11.7538             -1.4716     -27.7669       2.4840  EMA:80, X:36, PCT:0.33
2021     -25.8850           49  14.2857                5.6364             -1.5557     -55.3831      -0.4674  EMA:80, X:36, PCT:0.33
2022      39.4156           50  24.0000                8.6250             -1.6864     -23.8739       1.6510  EMA:80, X:36, PCT:0.33
2023      10.5058           35  22.8571                6.7567             -1

/tmp/ipykernel_59288/532239134.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio              Variation
2017            0            0      NaN                   NaN                 NaN          NaN          NaN  EMA:80, X:36, PCT:0.5
2018      28.6120           31  29.0323                6.9652             -1.5489     -20.0894       1.4242  EMA:80, X:36, PCT:0.5
2019      -5.1368           38  18.4211                6.3546             -1.6006     -25.7787      -0.1993  EMA:80, X:36, PCT:0.5
2020      70.7150           36  27.7778               10.9329             -1.4852     -23.6598       2.9888  EMA:80, X:36, PCT:0.5
2021     -11.3915           37  13.5135                6.7710             -1.4140     -39.9954      -0.2848  EMA:80, X:36, PCT:0.5
2022      11.8516           44  25.0000                5.8832             -1.6019     -24.7108       0.4796  EMA:80, X:36, PCT:0.5
2023       6.6602           23  26.0870                5.5802             -1.5777  

In [23]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
for x, y in sorted_stats.items():
    print(y.to_string())

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio              Variation
2017            0            0      NaN                   NaN                 NaN          NaN          NaN  EMA:74, X:4, PCT:0.25
2018      65.0632           35  34.2857                8.8457             -1.7863     -17.2641       3.7687  EMA:74, X:4, PCT:0.25
2019      10.0063           45  20.0000                9.7350             -2.1558     -41.8929       0.2389  EMA:74, X:4, PCT:0.25
2020     232.4926           44  27.2727               25.0259             -2.1193     -38.7377       6.0017  EMA:74, X:4, PCT:0.25
2021     -10.8233           52  19.2308                7.1488             -1.9598     -63.0929      -0.1715  EMA:74, X:4, PCT:0.25
2022       8.5685           53  18.8679               10.7421             -2.2989     -38.7080       0.2214  EMA:74, X:4, PCT:0.25
2023      11.2669           36  30.5556                5.3025             -1.8824  